In [5]:
# (c) 2014 Everaldo Aguiar & Reid Johnson
#
# Modified from:
# Marcel Caraciolo (https://gist.github.com/marcelcaraciolo/1423287)
#
# Functions to compute and extract association rules from a given frequent 
# itemset generated by the Apriori algorithm.

def apriori(dataset, min_support=0.5, verbose=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.

    References
    ----------
    .. [1] R. Agrawal, R. Srikant, "Fast Algorithms for Mining Association 
           Rules", 1994.

    """
    C1 = create_candidates(dataset)
    D = [set(r) for r in dataset]
    F1, support_data = support_prune(D, C1, min_support, verbose=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if verbose:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print("" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

def create_candidates(dataset, verbose=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if verbose:
        # Print a list of all the candidate items.
        print("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    #return map(frozenset, c1)
    return [frozenset(r) for r in c1]

def support_prune(dataset, candidates, min_support, verbose=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset)) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if verbose:
        for kset in retlist:
            for item in kset:
                print("{" + str(item) + "}")
        print("")
        for key in sscnt:
            print("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets being evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList

def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.
    """
    m = len(H[0])
    if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1, support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([itemset]) for itemset in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)

    return rules

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('2019-Oct.csv')

In [8]:
purchase_df = df[df['event_type'] == 'purchase']
purchase_df = purchase_df.sort_values(by=['user_session', 'event_time'])
purchase_sessions = purchase_df.groupby('user_session')['product_id'].apply(list).reset_index(name='products')
purchase_sessions = purchase_sessions[purchase_sessions['products'].apply(len) > 1]
purchase_sessions

,user_session,products
2,00004ada-8f93-49a6-956d-4ed71ae94791,"[1005031, 1005031]"
4,00005b76-13ba-4afe-b80d-2f2b337d3e92,"[1004806, 1005066]"
5,00007b3f-ec7a-4d2e-ac44-64dfec829806,"[12719135, 12719135]"
9,0000de39-dc74-414d-8da6-83ad56135bf5,"[1004246, 1002544, 1004767, 1004833]"
11,0000fa47-9577-480a-9fa4-be5c25e8dd59,"[1004767, 1004836]"
...,...,...
629511,fffabe59-a6ad-4159-ac40-c7f091c944c4,"[6600082, 6600082]"
629523,fffc0956-c9fb-402a-a8b6-80c833487834,"[1002544, 1004767]"
629526,fffc6f01-ba1f-4f96-af05-399cb2d6b39f,"[1004833, 1004833]"
629531,fffcc946-4682-4328-8d68-92d155112fd7,"[26400301, 5300097]"


In [9]:
dataset = purchase_sessions['products'].tolist()
dataset = [session for session in dataset if len(session) > 0]
dataset

[[1005031, 1005031],
 [1004806, 1005066],
 [12719135, 12719135],
 [1004246, 1002544, 1004767, 1004833],
 [1004767, 1004836],
 [1004767, 1004767],
 [1004776, 1004776],
 [1002633, 1002633, 1002633, 1002633, 1002633],
 [1004767, 1004767],
 [12100456, 6300739],
 [4700478, 1005141],
 [7002254, 26203478],
 [1003317, 1004249],
 [1802010, 1802010],
 [1005105, 1005115],
 [12703015, 12703015, 12703015],
 [1004856, 1004833],
 [5100816, 5100503],
 [1005015, 1005016],
 [26400224, 26400767],
 [1004833, 1004856, 1004856],
 [1003306, 1003306],
 [1005133, 26401046],
 [1005135, 1005106],
 [3800819, 3800819],
 [3600666, 3600666],
 [1004873, 1003535],
 [17300637, 17301187],
 [14500017, 1004768],
 [9400037, 9400037],
 [1004767, 1004209],
 [1004857, 1005098],
 [1002544, 1002544],
 [12703015, 1002629, 1004833],
 [26300510, 26300509, 26300508, 26300527],
 [1307004, 1305566, 1307408],
 [1004903, 1004209, 1004209],
 [1005207, 1004781],
 [1004249, 1004258, 1003316, 1004237, 1004250],
 [1004856, 1004856],
 [31501

In [10]:
F, support_data = apriori(dataset, min_support=0.001, verbose=True)

rules = generate_rules(F, support_data, min_confidence=0.1, verbose=True)
rules


{26300086}:  sup = 0.001
{1005102}:  sup = 0.001
{4803878}:  sup = 0.001
{5300157}:  sup = 0.001
{5100376}:  sup = 0.001
{3601437}:  sup = 0.001
{1004748}:  sup = 0.001
{1801555}:  sup = 0.002
{1005007}:  sup = 0.001
{1004433}:  sup = 0.002
{4100129}:  sup = 0.001
{3701056}:  sup = 0.001
{1307366}:  sup = 0.002
{1004887}:  sup = 0.001
{1005062}:  sup = 0.001
{2702277}:  sup = 0.001
{1004240}:  sup = 0.001
{1004751}:  sup = 0.002
{1002528}:  sup = 0.002
{1004259}:  sup = 0.002
{1005144}:  sup = 0.002
{1005073}:  sup = 0.002
{1004872}:  sup = 0.003
{1801766}:  sup = 0.003
{3801134}:  sup = 0.001
{3701134}:  sup = 0.001
{2701657}:  sup = 0.001
{1005072}:  sup = 0.001
{1801739}:  sup = 0.002
{3700766}:  sup = 0.001
{1004247}:  sup = 0.002
{1005169}:  sup = 0.001
{1004795}:  sup = 0.001
{1005217}:  sup = 0.001
{4804409}:  sup = 0.001
{1306359}:  sup = 0.001
{1004708}:  sup = 0.003
{1005006}:  sup = 0.003
{3700926}:  sup = 0.003
{1005124}:  sup = 0.002
{1005174}:  sup = 0.002
{1004720}:  sup

[(frozenset({1003306}), frozenset({1002544}), 0.13360323886639677),
 (frozenset({1005100}), frozenset({1004856}), 0.1287793952967525),
 (frozenset({4804055}), frozenset({4804056}), 0.18552875695732837),
 (frozenset({1004836}), frozenset({1004856}), 0.10950819672131148),
 (frozenset({1004750}), frozenset({1004856}), 0.1335282651072125),
 (frozenset({1004870}), frozenset({1004767}), 0.1415617128463476),
 (frozenset({1004750}), frozenset({1004833}), 0.1286549707602339),
 (frozenset({1002524}), frozenset({1002544}), 0.11172796668979873),
 (frozenset({1003317}), frozenset({1005115}), 0.12437810945273631),
 (frozenset({1004249}), frozenset({1005115}), 0.11431623931623931),
 (frozenset({1004750}), frozenset({1004767}), 0.14132553606237816),
 (frozenset({1004873}), frozenset({1004767}), 0.13194959229058562),
 (frozenset({1004209}), frozenset({1004856}), 0.23716012084592145),
 (frozenset({1004873}), frozenset({1004870}), 0.10971089696071164),
 (frozenset({1004858}), frozenset({1004856}), 0.1996

In [11]:
for rule in rules:
    antecedent, consequent, confidence = rule
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, confidence: {confidence}")

Rule: {1003306} -> {1002544}, confidence: 0.13360323886639677
Rule: {1005100} -> {1004856}, confidence: 0.1287793952967525
Rule: {4804055} -> {4804056}, confidence: 0.18552875695732837
Rule: {1004836} -> {1004856}, confidence: 0.10950819672131148
Rule: {1004750} -> {1004856}, confidence: 0.1335282651072125
Rule: {1004870} -> {1004767}, confidence: 0.1415617128463476
Rule: {1004750} -> {1004833}, confidence: 0.1286549707602339
Rule: {1002524} -> {1002544}, confidence: 0.11172796668979873
Rule: {1003317} -> {1005115}, confidence: 0.12437810945273631
Rule: {1004249} -> {1005115}, confidence: 0.11431623931623931
Rule: {1004750} -> {1004767}, confidence: 0.14132553606237816
Rule: {1004873} -> {1004767}, confidence: 0.13194959229058562
Rule: {1004209} -> {1004856}, confidence: 0.23716012084592145
Rule: {1004873} -> {1004870}, confidence: 0.10971089696071164
Rule: {1004858} -> {1004856}, confidence: 0.19969512195121952
Rule: {1004836} -> {1004833}, confidence: 0.11606557377049181
Rule: {10047

In [14]:
product_ids = [1004209, 1004856]
filtered_df = df[df['product_id'].isin(product_ids)][['product_id', 'category_code', 'brand']].drop_duplicates(subset='product_id')
filtered_df

,product_id,category_code,brand
79,1004856,electronics.smartphone,samsung
309,1004209,electronics.smartphone,samsung
